In [7]:
import os
import json
import numpy
import datetime
import requests 
import pandas as pd

import pymongo
from sqlalchemy import create_engine

In [8]:

mysql_uid = "gab8un"
mysql_pwd = "Passw0rd123"
mysql_url = "mysql-gab8un.mysql.database.azure.com"

atlas_cluster_name = "cluster0.q0atcnd"
atlas_user_name = "gab8un"
atlas_password = "XwlxqfpWD8PxodaA"

mongo_connections = {
    "local": f"mongodb://localhost:27017/",
    "atlas": f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net"
}

conn_str = mongo_connections['atlas']
source_db = 'sakila'
dest_db = 'sakila_dw'

In [9]:

def get_sql_dataframe(user_id, pwd, db_url, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{db_url}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe


def set_dataframe(user_id, pwd, db_url, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{db_url}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

## Load data from database

The majority of the data that is used comes from the MySQL database


In [19]:
rentals_sql = "SELECT * FROM sakila.rental;"
rentals_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, rentals_sql)
rentals_df.head(5)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [21]:
staff_sql = "SELECT * FROM sakila.staff;"
staff_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, staff_sql)
staff_df.head(5)

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-02-15 03:57:16
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,None,2006-02-15 03:57:16


In [24]:
stores_sql = "SELECT store_id, manager_staff_id, address FROM sakila.store AS s INNER JOIN sakila.address as a ON s.address_id = a.address_id;"
stores_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, stores_sql)
stores_df.head(5)

,store_id,manager_staff_id,address
0,1,1,47 MySakila Drive
1,2,2,28 MySQL Boulevard


## Retrieve data from remote sources
Data is stored in a mongoDB database and at an API endpoint

In [12]:
film_api_endpoint = "https://api.npoint.io/04c66e9121c9572e6442"
film_df = pd.read_json(film_api_endpoint)
film_df.head(5)

,title,length,rating,film_id,description,language_id,last_update,rental_rate,release_year,rental_duration,replacement_cost,special_features,original_language_id
0,ACADEMY DINOSAUR,86,PG,3,A Epic Drama of a Feminist And a Mad Scientist...,1,1139979822000,0.99,2006,6,20.99,"Deleted Scenes,Behind the Scenes",NaN
1,ACE GOLDFINGER,48,G,2,A Astounding Epistle of a Database Administrat...,1,1139979822000,4.99,2006,3,12.99,"Trailers,Deleted Scenes",NaN
2,ADAPTATION HOLES,50,NC-17,3,A Astounding Reflection of a Lumberjack And a ...,1,1139979822000,2.99,2006,7,18.99,"Trailers,Deleted Scenes",NaN
3,AFFAIR PREJUDICE,117,G,4,A Fanciful Documentary of a Frisbee And a Lumb...,1,1139979822000,2.99,2006,5,26.99,"Commentaries,Behind the Scenes",NaN
4,AFRICAN EGG,130,G,5,A Fast-Paced Documentary of a Pastry Chef And ...,1,1139979822000,2.99,2006,6,22.99,Deleted Scenes,NaN


In [14]:
query = {}
collection = "customers"
customers_df= get_mongo_dataframe(conn_str, source_db, collection, query)
customers_df.head(5)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,1139954676000,1139979440000
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,1139954676000,1139979440000
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,1139954676000,1139979440000
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,1139954676000,1139979440000
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,1139954676000,1139979440000


## Create Dimension Tables